In [6]:
#Importer le module xmltodic
import xmltodict
import pandas as pd
 
def create_dict_from_xml(chemin_fichier):
    with open(chemin_fichier, encoding='utf8') as fd:
        doc = xmltodict.parse(fd.read(), dict_constructor=dict)
    return doc

In [53]:
"""
key code : @name
    key enum : ['xs:attribute']['xs:simpleType']['xs:restriction']['xs:enumeration'][i]['@value']
    value enum : ['xs:attribute']['xs:simpleType']['xs:restriction']['xs:enumeration'][i]['xs:annotation']['xs:documentation']
"""
def parse_complex_type(annexe_type : dict):
    list_records = []
    all_complex_type = annexe_type['xs:schema']['xs:complexType']
    types_complexe = dict()
    i=0
    for complex_type in all_complex_type:
        temp_dict = dict()
        nom_type_complexe = complex_type['@name']
        if isinstance(complex_type['xs:attribute'], dict):
            for element in complex_type['xs:attribute']['xs:simpleType']['xs:restriction']['xs:enumeration']:
                temp_dict[element['@value']] = element.get('xs:annotation', {}).get('xs:documentation', element['@value'])
        
        types_complexe[nom_type_complexe] = temp_dict
        result_dict = {
            "type" : nom_type_complexe,
            "enum" : temp_dict
        }
        list_records.append(result_dict)
    
    df = pd.DataFrame.from_records(list_records)

    return df, types_complexe

def generate_complex_type_df(chemin : str) -> pd.DataFrame:
    annexe_type = create_dict_from_xml(chemin)
    complexe_types_df, complexe_types_dict = parse_complex_type(annexe_type)
    return complexe_types_df

def generate_complex_type_dict(chemin : str) -> pd.DataFrame:
    annexe_type = create_dict_from_xml(chemin)
    complexe_types_df, complexe_types_dict = parse_complex_type(annexe_type)
    return complexe_types_dict

In [8]:
#annexe_type = create_dict_from_xml("/home/admin/projet/dataprep-ab/download/SchemaDocBudg/CommunAnnexe.xsd")
#annexe_complexe_types = parse_complex_type(annexe_type)
annexe_complexe_types = generate_complex_type_df("/home/admin/projet/dataprep-ab/download/SchemaDocBudg/CommunAnnexe.xsd")
annexe_complexe_types


,type,enum
0,ATCodNatOrgGroup,"{'01': 'EPCI', '02': 'Syndicats mixtes (articl..."
1,ATCodInvFonc,"{'I': 'Investissement', 'F': 'Fonctionnement'}"
2,ATCodTypAgent,"{'I': 'Titulaire', 'N': 'Non titulaires', 'R':..."
3,ATCodCatAgent,"{'A': 'A', 'B': 'B', 'C': 'C'}"
4,ATNatureContrat,"{'CDD': 'Contrat à durée déterminée', 'CDI': '..."
...,...,...
87,ATCodTypeCharge,{'01': 'Charges de fonctionnement à répartir (...
88,ATCodTypeChapitreAPCP,"{'1': 'Chapitre nature', '2': 'Chapitre opérat..."
89,ATOrigineRecette,"{'1': 'Part régionale des ressources', '2': 'P..."
90,ATCodTypeGestion,"{'1': 'AP/AE', '2': 'Crédits de paiement'}"


In [57]:
"""
nom du champ : '@name'
type du champ : '@type'
libelle : ['xs:annotation']['xs:annotation']['z:libelle']
description : ['xs:annotation']['xs:annotation']['z:description']
"""
def _init_annexe_data_dictionnary(class_annexe : dict) -> pd.DataFrame:
    elements = class_annexe['xs:sequence']['xs:element']
    list_records = []
    dict_champs = dict()
    nom_annexe = class_annexe["@name"][1:]
    for element in elements:
        documentation = element['xs:annotation']['xs:documentation']
        if isinstance(documentation, str):
            libelle = documentation
            description = documentation
        else:
            libelle = element['xs:annotation']['xs:documentation']['z:libelle']
            description = element['xs:annotation']['xs:documentation'].get('z:description')
        dict_champs = {
            "nom_annexe" : nom_annexe,
            "nom-champ" : element["@name"],
            "type" : element["@type"],
            "libelle" : libelle,
            "description" : description,
        }
        list_records.append(dict_champs)
    
    df = pd.DataFrame.from_records(list_records)
    df["description"] = df["description"].str.replace(r'^<[^<>]*>', '', regex=True)
    df["description"] = df["description"].str.replace(r'^\s*<ul>', '', regex=True)
    df["description"] = df["description"].str.replace(r'^\s*<li>', '', regex=True)
    df["description"] = df["description"].str.replace(r'<ul>', ' : ', regex=True)
    df["description"] = df["description"].str.replace(r'<li>', ' - ', regex=True)
    df["description"] = df["description"].str.replace(r'<[^<>]*>', ' ', regex=True)
    df["description"] = df["description"].str.replace(r'\s\s+', ' ', regex=True)

    
    return df

def generate_annexe_data_document(chemin_annexe: str, complex_type_df: pd.DataFrame) -> pd.DataFrame:
    class_to_generate = create_dict_from_xml(chemin_annexe)['xs:schema']['xs:complexType'][1]
    init_df = _init_annexe_data_dictionnary(class_to_generate)
    init_df = init_df.merge(complex_type_df, how='left')
    return init_df

In [51]:
class_emprunt = create_dict_from_xml("/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_Formation_Pro_Jeunes.xsd")['xs:schema']['xs:complexType'][1]
init_df = _init_annexe_data_dictionnary(class_emprunt)
init_df

,nom_annexe,nom-champ,type,libelle,description
0,FormationProJeunes,CodRDTot,ATCodRD,Sens,Sens
1,FormationProJeunes,CodRessExt,ATCodRessExt,Ressource Externe,Obligatoire si Dépense
2,FormationProJeunes,CodApprent,ATCodApprent,Apprentissage,Si Dépense : - Apprentissage - Enseignement Pr...
3,FormationProJeunes,MtFormN,Base_Montant,Montant formation exercice N,Montant formation exercice N
4,FormationProJeunes,MtFormN_1,Base_Montant,Montant formation exercice N-1,Montant formation exercice N-1
5,FormationProJeunes,Champ_Editeur,Base_Texte100,Commentaire,Commentaire


In [52]:
class_emprunt = create_dict_from_xml("/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_Emprunt.xsd")['xs:schema']['xs:complexType'][1]
df = _init_annexe_data_dictionnary(class_emprunt)
df.merge(annexe_complexe_types, how='left')

generate_annexe_data_document("/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_Pret.xsd", annexe_complexe_types)

,nom_annexe,nom-champ,type,libelle,description,enum
0,Pret,CodTypPret,ATCodTypPret,Type prêt,Type prêt,"{'A': 'Assortit d'intérêt', 'N': 'Non assortit..."
1,Pret,NomBenefPret,Base_Texte100,Nom du bénéficiaire du prêt,Nom du bénéficiaire du prêt,NaN
2,Pret,DtDelib,Base_Date,Date de Délibération,Date de Délibération,NaN
3,Pret,MtCapitalRestDu_01_01,Base_Montant,Capital restant dû au 01/01 ou début exercice,Capital restant dû au 01/01 ou début exercice,NaN
4,Pret,MtCapitalRestDu_31_12,Base_Montant,Capital restant dû au 31/12 ou fin exercice,Capital restant dû au 31/12 ou fin exercice,NaN
5,Pret,MtCapitalExer,Base_Montant,Montant capital à rembourser dans l'exercice,Montant capital à rembourser dans l'exercice,NaN
6,Pret,MtIntExer,Base_Montant,Montant intérêt à verser dans l'exercice,Montant intérêt à verser dans l'exercice,NaN
7,Pret,MtICNE,Base_Montant,ICNE exercice,ICNE exercice,NaN
8,Pret,Champ_Editeur,Base_Texte100,Commentaire,Commentaire,NaN


In [44]:
dict_annexe = create_dict_from_xml("/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_Annexes.xsd")["xs:schema"]['xs:include']
dict_annexe.pop(0)
class_annexe_paths = []
for annexe in dict_annexe: 
    class_annexe_paths.append(f"/home/admin/projet/dataprep-ab/download/SchemaDocBudg/{annexe['@schemaLocation']}")
class_annexe_paths


['/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_Emprunt.xsd',
 '/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_Tresorerie.xsd',
 '/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_Charges.xsd',
 '/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_Tiers.xsd',
 '/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_CreditBail.xsd',
 '/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_PPP.xsd',
 '/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_AutresEngagements.xsd',
 '/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_Concours.xsd',
 '/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_RecettesAffectees.xsd',
 '/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_Formation.xsd',
 '/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_Fiscalite.xsd',
 '/home/admin/projet/dataprep-ab/download/SchemaDocBudg/Class_Consolidation.xsd',
 '/home/admin/projet/dataprep-ab/download/SchemaDocBudg/C

In [54]:
df_result = pd.DataFrame()

for annexe_path in class_annexe_paths:
    df = generate_annexe_data_document(annexe_path, annexe_complexe_types)
    df_result = pd.concat([df, df_result])

In [60]:
annexe_dict = generate_complex_type_dict("/home/admin/projet/dataprep-ab/download/SchemaDocBudg/CommunAnnexe.xsd")
annexe_dict

{'ATCodNatOrgGroup': {'01': 'EPCI',
  '02': 'Syndicats mixtes (article 5721-2 du CGCT)',
  '03': 'Syndicats mixtes (article L5721-1 du CGCT)',
  '09': 'Autres'},
 'ATCodInvFonc': {'I': 'Investissement', 'F': 'Fonctionnement'},
 'ATCodTypAgent': {'I': 'Titulaire',
  'N': 'Non titulaires',
  'R': 'Rattachement'},
 'ATCodCatAgent': {'A': 'A', 'B': 'B', 'C': 'C'},
 'ATNatureContrat': {'CDD': 'Contrat à durée déterminée',
  'CDI': 'Contrat à durée indéterminée',
  'A': 'Autres'},
 'ATCodSectAgentTitulaire': {'ADM': 'Filière administrative',
  'TECH': 'Technique',
  'S': 'Sociale',
  'MS': 'Médico-sociale',
  'MT': 'Médico-technique',
  'SP': 'Sportive',
  'CULT': 'Culturelle',
  'ANIM': 'Animation',
  'POL': 'Police',
  'X': 'Emplois non cités',
  'DIR1': 'Directeur général des services',
  'DIR2': 'Directeur général adjoint des services',
  'DIR3': 'Collaborateur de cabinet (ne plus utiliser à compter de 2013)',
  'DIR4': 'Directeur général des services techniques',
  'DIR5': "Emplois créé

In [35]:

df_result.to_csv('./csv/dict_donnees.csv', index=False)  

In [47]:
df_result

,nom_annexe,nom-champ,type,libelle,description,enum
0,FluxCroise,CodTypFlux,ATCodTypFlux,Indicateur type de flux,None,{'01': 'Flux réciproques entre le groupement à...
1,FluxCroise,CodInvFonc,ATCodInvFonc,Section,None,"{'I': 'Investissement', 'F': 'Fonctionnement'}"
2,FluxCroise,CodRD,ATCodRD,Sens,None,"{'R': 'Recette', 'D': 'Dépense'}"
3,FluxCroise,MtCredOuv,Base_Montant,Crédits ouverts,None,NaN
4,FluxCroise,MtReal,Base_Montant,Réalisations,None,NaN
...,...,...,...,...,...,...
92,Emprunt,MtEmprReneg,Base_Montant,Montant du contrat de l'emprunt renégocié,Montant du contrat de l'emprunt renégocié,NaN
93,Emprunt,MtCRDRefin,Base_Montant,Capital restant dû au moment du refinancement,A renseigner pour les emprunts refinancés et d...,NaN
94,Emprunt,MtCapitalReamenage,Base_Montant,Capital réaménagé,A renseigner pour les emprunts refinancés et d...,NaN
95,Emprunt,Champ_Editeur,Base_Texte100,Commentaire,Donnée libre,NaN
